In [ ]:
# !pip install leidenalg

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, calinski_harabasz_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import time
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score
import pickle
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
# category = "real_data"
for category in [  "balanced_data", "imbalanced_data", "real_data",
                ]:

    path= ".."
    if category in ["balanced_data", "imbalanced_data"]:
        files = glob2.glob(f'{path}/R/simulated_data/{category}/*.h5')
        files = [f[len(f"{path}/R/simulated_data/{category}/"):-3] for f in files]
    else:
        files = glob2.glob(f'{path}/real_data/*.h5')
        files = [f[len(f"{path}/real_data/"):-3] for f in files]
    print(files)

    df = pd.DataFrame(columns = ["dataset", "ARI", "NMI", "sil", "run", "time", "pred", "cal"])
    for dataset in files:
        if category in ["balanced_data", "imbalanced_data"]:
            data_mat = h5py.File(f"{path}/R/simulated_data/{category}/{dataset}.h5","r")
        else:
            data_mat = h5py.File(f"{path}/real_data/{dataset}.h5","r")

        Y = np.array(data_mat['Y'])
        X = np.array(data_mat['X'])
        print(f">>>>dataset {dataset}")
        if X.shape[0] > 10000:
            continue

        X = np.ceil(X).astype(np.int)
        for run in range(3):
            start = time.time()
            adata = sc.AnnData(X)
            adata.obs['Group'] = Y
            adata.var_names_make_unique()

            sc.pp.filter_genes(adata, min_cells=3)

            sc.pp.normalize_total(adata, target_sum=1e4)

            sc.pp.log1p(adata)

            sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

            adata = adata[:, adata.var.highly_variable]

            sc.pp.scale(adata, max_value=10)

            sc.tl.pca(adata, svd_solver='arpack')

            sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

            sc.tl.umap(adata)

            sc.tl.leiden(adata)

            pred = adata.obs['leiden'].to_list()
            pred = [int(x) for x in pred]

            elapsed = time.time() - start
            ARI = adjusted_rand_score(Y, pred)
            NMI = np.around(normalized_mutual_info_score(Y, pred), 5)
            ss = silhouette_score(adata.obsm["X_umap"],pred)
            cal = calinski_harabasz_score(adata.obsm["X_umap"],pred)

            df.loc[df.shape[0]] = [dataset, ARI, NMI, ss, run,elapsed, pred, cal]
            df.to_pickle(f"../output/pickle_results/{category}/{category}_scanpy.pkl")


['data_1c8', 'data_-1c4', 'data_-1c8', 'data_0c4', 'data_0c8', 'data_0c16', 'data_1.5c4', 'data_1c4', 'data_1.5c8', 'data_1.5c16', 'data_-1c16', 'data_1c16']
>>>>dataset data_1c8


/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


>>>>dataset data_-1c4


/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


>>>>dataset data_-1c8


...

/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


>>>>dataset Muraro


/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


>>>>dataset 10X_PBMC


/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:848: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
